## Preambule

In [4]:
# Plotting
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import xarray as xr
from plotly.subplots import make_subplots
from tqdm import tqdm

## Input

In [5]:
rules_b = ["GF", "PC", "PCC", "AP", "GDR", "ECPC"]
rules_a = rules_b

## Read data

In [6]:
xr_dataread = xr.open_dataset(
    "K:/Data/Data_effortsharing/DataUpdate_ongoing/startyear_2021/xr_dataread.nc"
)
# xr_temps = xr.open_dataset("K:/Data/Data_effortsharing/DataUpdate_ongoing/startyear_2021/xr_temps.nc")
# xr_2030 = xr.open_dataset("X:/user/dekkerm/Projects/ECEMF_T5.2/Data/xr_alloc_2030.nc")

In [7]:
df_eng = pd.read_csv(
    "X:/user/dekkerm/Data/ENGAGE/PolicyScenarios/engage-internal_snapshot_1695642798.csv"
)
df_eng = df_eng[df_eng.Variable == "Emissions|Kyoto Gases"]
df_eng = df_eng.reset_index(drop=True)
regions_df = np.array(df_eng.Region)
regions_df[regions_df == "Argentine Republic"] = "ARG"
regions_df[regions_df == "Canada"] = "CAN"
regions_df[regions_df == "Commonwealth of Australia"] = "AUS"
regions_df[regions_df == "Federative Republic of Brazil"] = "BRA"
regions_df[regions_df == "People's Repulic of China"] = "CHN"
regions_df[regions_df == "European Union (28 member countries)"] = "EU"
regions_df[regions_df == "Republic of India"] = "IND"
regions_df[regions_df == "Republic of Indonesia"] = "IDN"
regions_df[regions_df == "State of Japan"] = "JPN"
regions_df[regions_df == "Russian Federation"] = "RUS"
regions_df[regions_df == "Kingdom of Saudi Arabia"] = "SAU"
regions_df[regions_df == "Republic of South Africa"] = "ZAF"
regions_df[regions_df == "Republic of Korea (South Korea)"] = "KOR"
regions_df[regions_df == "United Mexican States"] = "MEX"
regions_df[regions_df == "Republic of Turkey"] = "TUR"
regions_df[regions_df == "United States of America"] = "USA"
regions_df[regions_df == "Viet Nam "] = "VNM"
df_eng.Region = regions_df
df = pd.read_excel(
    "X:/user/dekkerm/Data//UNFCCC_Parties_Groups_noeu.xlsx", sheet_name="Country groups"
)
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [8]:
curpol = "GP_CurPol_T45"
ndc = "GP_NDC2030_T45"
nz = "GP_Glasgow"
df_eng_ref = df_eng[["Model", "Scenario", "Region"] + list(df_eng.keys()[5:])]
df_eng_ref = df_eng_ref[df_eng_ref.Scenario.isin([curpol, ndc, nz])]
scen = np.array(df_eng_ref.Scenario)
scen[scen == ndc] = "NDC"
scen[scen == curpol] = "CurPol"
scen[scen == nz] = "NetZero"
reg = np.array(df_eng_ref.Region)
reg[reg == "World"] = "WORLD"
df_eng_ref["Scenario"] = scen
df_eng_ref["Region"] = reg
dummy = df_eng_ref.melt(
    id_vars=["Scenario", "Model", "Region"], var_name="Time", value_name="Value"
)
dummy["Time"] = np.array(dummy["Time"].astype(int))
dummy = dummy.set_index(["Scenario", "Model", "Region", "Time"])
xr_eng = xr.Dataset.from_dataframe(dummy)
xr_eng = xr_eng.reindex(Time=np.arange(1850, 2101))
xr_eng = xr_eng.interpolate_na(dim="Time", method="linear")

## Read memberstate info

In [9]:
for cty_i, cty in tqdm(enumerate(group_eu)):
    ds = (
        xr.open_dataset(
            "K:/Data/Data_effortsharing/DataUpdate_ongoing/startyear_2021/Allocations_GHG_incl/"
            + "xr_alloc_"
            + cty
            + ".nc"
        )
        .sel(Temperature=1.5, Risk=0.5)
        .expand_dims(Region=[cty])
    )
    if cty_i == 0:
        xrt = ds.copy()
    else:
        xrt = xr.merge([xrt, ds])
    ds.close()
xrt = xrt.astype("float32")

27it [00:48,  1.78s/it]


In [10]:
xrt

<xarray.Dataset> Size: 165MB
Dimensions:               (Region: 27, NegEmis: 7, NonCO2red: 5, Timing: 2,
                           Time: 80, Scenario: 5, Convergence_year: 13,
                           Discount_factor: 4, Historical_startyear: 3,
                           Capability_threshold: 3, RCI_weight: 3)
Coordinates: (12/14)
  * Scenario              (Scenario) <U4 80B 'SSP1' 'SSP2' 'SSP3' 'SSP4' 'SSP5'
  * Time                  (Time) int32 320B 2021 2022 2023 ... 2098 2099 2100
  * NonCO2red             (NonCO2red) float64 40B 0.1 0.33 0.5 0.67 0.9
  * NegEmis               (NegEmis) float64 56B 0.2 0.3 0.4 0.5 0.6 0.7 0.8
  * Timing                (Timing) <U9 72B 'Immediate' 'Delayed'
  * Convergence_year      (Convergence_year) int32 52B 2040 2045 ... 2095 2100
    ...                    ...
  * Capability_threshold  (Capability_threshold) <U4 48B 'No' 'PrTh' 'Th'
  * RCI_weight            (RCI_weight) <U4 48B 'Cap' 'Half' 'Resp'
  * Region                (Region) object 216B 'AUT' 'BEL' 'BGR' ... 'SVN' 'SWE'
    Temperature           float64 8B 1.5
    Risk                  float64 8B 0.5
    Variable              <U24 96B 'Emissions|N2O|AFOLU|Land'
Data variables:
    GF                    (Region, NegEmis, NonCO2red, Timing, Time) float32 605kB ...
    PC                    (Region, Scenario, NegEmis, NonCO2red, Timing, Time) float32 3MB ...
    PCC                   (Region, Time, Convergence_year, NegEmis, NonCO2red, Timing, Scenario) float32 39MB ...
    ECPC                  (Region, Discount_factor, Historical_startyear, NegEmis, NonCO2red, Timing, Time, Scenario) float32 36MB ...
    AP                    (Region, Scenario, Time, NegEmis, NonCO2red, Timing) float32 3MB ...
    GDR                   (Region, Scenario, Time, NegEmis, NonCO2red, Timing, Historical_startyear, Capability_threshold, RCI_weight) float32 82MB ...
    PCB                   (Region, NegEmis, NonCO2red, Timing, Time) float32 605kB ...
    PCB_lin               (Region, NegEmis, NonCO2red, Timing, Time) float32 605kB ...

## Plot

In [11]:
rulecolors = [
    "sienna",
    "goldenrod",
    "forestgreen",
    "mediumvioletred",
    "steelblue",
    "tomato",
    "silver",
]
rules = ["GF", "PC", "PCC", "ECPC", "AP", "GDR"]
standard_settings = {
    "NonCO2red": 0.5,
    "Timing": "Immediate",
    "NegEmis": 0.4,
    "Convergence_year": 2050,
    "Discount_factor": 0,
    "Historical_startyear": 1990,
    "Capability_threshold": "Th",
    "RCI_weight": "Half",
}

In [12]:
for yearprev in [1990, 2021]:
    for year in [2030, 2040]:
        data_2030 = xrt.sel(**standard_settings, Time=year, Region=group_eu).mean(dim="Scenario")
        data_1990 = xr_dataread.sel(Time=yearprev).GHG_hist
        data_red = (data_1990 - data_2030) / data_1990

        fig = make_subplots(
            rows=2,
            cols=3,  # subplot_titles=('Grandfathering (GF)',
            # 'Per Capita (PC)',
            # 'Per Capita Convergence (PCC)',
            # 'Equal Cumulative Per Capita',
            # 'Ability to pay (AP)',
            # 'Greenhouse Development Rights'),
            specs=[
                [{"type": "choropleth"}, {"type": "choropleth"}, {"type": "choropleth"}],
                [{"type": "choropleth"}, {"type": "choropleth"}, {"type": "choropleth"}],
            ],
            horizontal_spacing=0,
            vertical_spacing=0,
        )

        colorbar_abs = dict(len=1, x=1, tickvals=[0, 1, 2, 3], ticktext=["1", "10", "100", "1000"])
        colorbar_pc = dict(len=1, x=1)

        for var_i, var in enumerate(rules):
            if var_i == 0:
                cm = True
            else:
                cm = False
            fig.add_trace(
                go.Choropleth(
                    locations=group_eu,
                    z=data_red[var],
                    locationmode="ISO-3",
                    colorscale="RdYlBu_r",
                    zmax=1.01,
                    zmin=0.29,
                    text=[
                        str(r)
                        + f": {float(data_red.sel(Region=r)[var]):0.2f} Gt CO<sub>2</sub>e"
                        for r_i, r in enumerate(group_eu)
                    ],
                    hovertemplate="%{text}",
                    name="",
                    marker_line_color="grey",
                    marker_line_width=0.7,
                    colorbar=dict(
                        len=1,
                        x=1,
                        tickvals=np.arange(0.3, 1, 0.1),
                        ticktext=[str(int(i * 100)) + "%" for i in np.arange(0.3, 1, 0.1)],
                        title={
                            "text": "Emission<br>reduction<br>in "
                            + str(year)
                            + "<br>w.r.t. "
                            + str(yearprev)
                        },
                        tickfont={"size": 25},
                    ),
                    showscale=cm,
                ),
                [1, 1, 1, 2, 2, 2][var_i],
                [1, 2, 3, 1, 2, 3][var_i],
            )
        # fig.update_geos(showocean=True, oceancolor="white")
        # fig.update_geos(showlakes=True, lakecolor="white")

        # limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
        fig.update_layout(
            geo1=dict(
                scope="europe",
                showlakes=False,
                lakecolor="rgb(255, 255, 255)",
                showcoastlines=False,
            ),
            geo2=dict(
                scope="europe",
                showlakes=False,
                lakecolor="rgb(255, 255, 255)",
                showcoastlines=False,
            ),
            geo3=dict(
                scope="europe",
                showlakes=False,
                lakecolor="rgb(255, 255, 255)",
                showcoastlines=False,
            ),
            geo4=dict(
                scope="europe",
                showlakes=False,
                lakecolor="rgb(255, 255, 255)",
                showcoastlines=False,
            ),
            geo5=dict(
                scope="europe",
                showlakes=False,
                lakecolor="rgb(255, 255, 255)",
                showcoastlines=False,
            ),
            geo6=dict(
                scope="europe",
                showlakes=False,
                showcoastlines=False,
            ),
        )

        mn = np.min(np.array([float(data_red[rules].min()[r]) for r in rules]))
        mx = np.max(np.array([float(data_red[rules].max()[r]) for r in rules]))
        for var_i, var in enumerate(rules):
            vals = np.array(data_red[var]).astype(float)
            vals_i = (vals + mn) ** 1.2
            vals_n = 14 * (vals - mn) / (mx - mn) + 7
            vals_boolean = np.copy(vals)
            vals_boolean[vals >= 0] = 1
            vals_boolean[vals < 0] = 0
            vals_boolean = vals_boolean.astype(int)
            vals[np.isnan(vals)] = np.nanmean(vals)

            fig.add_scattergeo(
                locations=group_eu,
                text=[str(int(vals[r_i] * 100)) + "%" for r_i in range(len(group_eu))],
                featureidkey="properties.NAME_3",
                mode="text",
                hoverinfo="skip",
                showlegend=False,
                textfont={"size": 20, "color": "black"},
                row=[1, 1, 1, 2, 2, 2][var_i],
                col=[1, 2, 3, 1, 2, 3][var_i],
            )

        fig.update_geos(
            visible=False,
            showlakes=True,
            lakecolor="rgb(255, 255, 255)",
            projection_type="natural earth",
            resolution=50,
            lataxis_range=[30, 75],
            lonaxis_range=[-20, 40],
            showcoastlines=False,
        )

        # fig.update_traces(showscale=False)
        fig.update_layout(height=1200, width=2300)
        fig.update_geos(visible=True, showcountries=False)  # fitbounds="locations",
        fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

        fig.add_annotation(
            xref="paper",
            yref="paper",
            ax=0,
            align="left",
            ay=-0,
            showarrow=False,
            font=dict(color="black", size=25),
            y=0.96,
            x=0.07,
            text="<b>(a) Grandfathering</b>",
        )
        fig.add_annotation(
            xref="paper",
            yref="paper",
            ax=0,
            align="left",
            ay=-0,
            showarrow=False,
            font=dict(color="black", size=25),
            y=0.96,
            x=0.47,
            text="<b>(b) Per Capita</b>",
        )
        fig.add_annotation(
            xref="paper",
            yref="paper",
            ax=0,
            align="left",
            ay=-0,
            showarrow=False,
            font=dict(color="black", size=25),
            y=0.96,
            x=0.90,
            text="<b>(c) Per Capita Convergence</b>",
        )
        fig.add_annotation(
            xref="paper",
            yref="paper",
            ax=0,
            align="left",
            ay=-0,
            showarrow=False,
            font=dict(color="black", size=25),
            y=0.45,
            x=0.07,
            text="<b>(d) Equal Cumulative per Capita</b>",
        )
        fig.add_annotation(
            xref="paper",
            yref="paper",
            ax=0,
            align="left",
            ay=-0,
            showarrow=False,
            font=dict(color="black", size=25),
            y=0.45,
            x=0.48,
            text="<b>(e) Ability to Pay</b>",
        )
        fig.add_annotation(
            xref="paper",
            yref="paper",
            ax=0,
            align="left",
            ay=-0,
            showarrow=False,
            font=dict(color="black", size=25),
            y=0.45,
            x=0.95,
            text="<b>(f) Greenhouse Development Rights</b>",
        )
        fig_europe = fig
        fig.show()
        fig.write_image(
            "K:/Code/EffortSharing/Figures/Paper_Europe/ReductionsMapEurope_"
            + str(year)
            + "_"
            + str(yearprev)
            + ".png",
            scale=3,
        )

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_8640\1970047442.py:81: RuntimeWarning:

invalid value encountered in cast



C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_8640\1970047442.py:81: RuntimeWarning:

invalid value encountered in cast



C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_8640\1970047442.py:81: RuntimeWarning:

invalid value encountered in cast



C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_8640\1970047442.py:81: RuntimeWarning:

invalid value encountered in cast



In [13]:
fig.write_image("K:/Code/EffortSharing/Figures/Paper_Europe/Europe_2040perc.png", scale=3)

In [14]:
data5 = xr_temps.sel(**standard_settings, Risk=0.5, Region=group_eu).mean(
    dim=["Conditionality", "Hot_air", "Ambition", "Scenario"]
)

NameError: name 'xr_temps' is not defined

In [ ]:
fig = make_subplots(
    rows=2,
    cols=3,  # subplot_titles=('Grandfathering (GF)',
    # 'Per Capita (PC)',
    # 'Per Capita Convergence (PCC)',
    # 'Equal Cumulative Per Capita',
    # 'Ability to pay (AP)',
    # 'Greenhouse Development Rights'),
    specs=[
        [{"type": "choropleth"}, {"type": "choropleth"}, {"type": "choropleth"}],
        [{"type": "choropleth"}, {"type": "choropleth"}, {"type": "choropleth"}],
    ],
    horizontal_spacing=0,
    vertical_spacing=0,
)

colorbar_abs = dict(len=1, x=1, tickvals=[0, 1, 2, 3], ticktext=["1", "10", "100", "1000"])
colorbar_pc = dict(len=1, x=1)

for var_i, var in enumerate(rules):
    if var_i == 0:
        cm = True
    else:
        cm = False
    fig.add_trace(
        go.Choropleth(
            locations=group_eu,
            z=data5[var],
            locationmode="ISO-3",
            colorscale="RdYlGn_r",
            zmax=2.5,
            zmin=1.5,
            text=[
                str(r) + f": {float(data5.sel(Region=r)[var]):0.2f} deg C"
                for r_i, r in enumerate(group_eu)
            ],
            hovertemplate="%{text}",
            name="",
            marker_line_color="grey",
            marker_line_width=0.7,
            colorbar=dict(
                len=1,
                x=1,
                tickvals=np.arange(1.5, 2.5, 0.1),
                ticktext=[str(np.round(i, 1)) + "&deg;C" for i in np.arange(1.5, 2.5, 0.1)],
                title={"text": "Temp-equiv<br>of NDC"},
                tickfont={"size": 25},
            ),
            showscale=cm,
        ),
        [1, 1, 1, 2, 2, 2][var_i],
        [1, 2, 3, 1, 2, 3][var_i],
    )
# fig.update_geos(showocean=True, oceancolor="white")
# fig.update_geos(showlakes=True, lakecolor="white")

# limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
fig.update_layout(
    geo1=dict(
        scope="europe",
        showlakes=False,
        lakecolor="rgb(255, 255, 255)",
        showcoastlines=False,
    ),
    geo2=dict(
        scope="europe",
        showlakes=False,
        lakecolor="rgb(255, 255, 255)",
        showcoastlines=False,
    ),
    geo3=dict(
        scope="europe",
        showlakes=False,
        lakecolor="rgb(255, 255, 255)",
        showcoastlines=False,
    ),
    geo4=dict(
        scope="europe",
        showlakes=False,
        lakecolor="rgb(255, 255, 255)",
        showcoastlines=False,
    ),
    geo5=dict(
        scope="europe",
        showlakes=False,
        lakecolor="rgb(255, 255, 255)",
        showcoastlines=False,
    ),
    geo6=dict(
        scope="europe",
        showlakes=False,
        showcoastlines=False,
    ),
)

mn = np.min(np.array([float(data5[rules].min()[r]) for r in rules]))
mx = np.max(np.array([float(data5[rules].max()[r]) for r in rules]))
for var_i, var in enumerate(rules):
    vals = np.array(data5[var]).astype(float)
    vals_i = (vals + mn) ** 1.2
    vals_n = 14 * (vals - mn) / (mx - mn) + 7
    vals_boolean = np.copy(vals)
    vals_boolean[vals >= 0] = 1
    vals_boolean[vals < 0] = 0
    vals_boolean = vals_boolean.astype(int)

    fig.add_scattergeo(
        locations=group_eu,
        text=[str(np.round(vals[r_i], 1)) for r_i in range(len(group_eu))],
        featureidkey="properties.NAME_3",
        mode="text",
        hoverinfo="skip",
        showlegend=False,
        textfont={"size": 20, "color": "black"},
        row=[1, 1, 1, 2, 2, 2][var_i],
        col=[1, 2, 3, 1, 2, 3][var_i],
    )

fig.update_geos(
    visible=False,
    showlakes=True,
    lakecolor="rgb(255, 255, 255)",
    projection_type="natural earth",
    resolution=50,
    lataxis_range=[30, 75],
    lonaxis_range=[-20, 40],
    showcoastlines=False,
)

# fig.update_traces(showscale=False)
fig.update_layout(height=1200, width=2300)
fig.update_geos(visible=True, showcountries=False)  # fitbounds="locations",
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

fig.add_annotation(
    xref="paper",
    yref="paper",
    ax=0,
    align="left",
    ay=-0,
    showarrow=False,
    font=dict(color="black", size=25),
    y=0.96,
    x=0.07,
    text="<b>(a) Grandfathering</b>",
)
fig.add_annotation(
    xref="paper",
    yref="paper",
    ax=0,
    align="left",
    ay=-0,
    showarrow=False,
    font=dict(color="black", size=25),
    y=0.96,
    x=0.47,
    text="<b>(b) Per Capita</b>",
)
fig.add_annotation(
    xref="paper",
    yref="paper",
    ax=0,
    align="left",
    ay=-0,
    showarrow=False,
    font=dict(color="black", size=25),
    y=0.96,
    x=0.90,
    text="<b>(c) Per Capita Convergence</b>",
)
fig.add_annotation(
    xref="paper",
    yref="paper",
    ax=0,
    align="left",
    ay=-0,
    showarrow=False,
    font=dict(color="black", size=25),
    y=0.45,
    x=0.07,
    text="<b>(d) Equal Cumulative per Capita</b>",
)
fig.add_annotation(
    xref="paper",
    yref="paper",
    ax=0,
    align="left",
    ay=-0,
    showarrow=False,
    font=dict(color="black", size=25),
    y=0.45,
    x=0.48,
    text="<b>(e) Ability to Pay</b>",
)
fig.add_annotation(
    xref="paper",
    yref="paper",
    ax=0,
    align="left",
    ay=-0,
    showarrow=False,
    font=dict(color="black", size=25),
    y=0.45,
    x=0.95,
    text="<b>(f) Greenhouse Development Rights</b>",
)
fig_europe = fig
fig.show()

In [ ]:
fig.write_image("K:/Code/EffortSharing/Figures/Paper_Europe/Europe_temp.png", scale=3)